# Importing the dataset

In [1]:
import requests 
import time
import json
import pandas as pd

## Get the api response

In [3]:
URL = "https://americas.api.riotgames.com"

def get_summoner_id(summoner_name, api_key):
    url = f'{URL}/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def get_match_list(api_key, summoner_id, start=0, count=10):
    url = f'{URL}/lol/match/v5/matches/by-puuid/{summoner_id}/ids?start={start}&count={count}&api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def get_match_data(match_id, api_key):
    url = f'{URL}/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def extract_match_info(match_data):
    game_duration = match_data['info']['gameDuration']
    game_mode = match_data['info']['gameMode']
    participants = match_data['info']['participants']
    
    participants_df = pd.DataFrame()
    
    for participant in participants:
        participant_info = {
            'championName': participant['championName'],
            'kills': participant['kills'],
            'death': participant['deaths'],
            'assists': participant['assists'],
            'totalDamageToChampions': participant['totalDamageDealtToChampions'],
            'goldEarned': participant['goldEarned'],
            'visioScore': participant['visionScore'],
            'role': participant['individualPosition'],
            'teamId': participant['teamId']
        }
        participants_df = pd.concat([participants_df, pd.DataFrame([participant_info])], ignore_index=True)
      
        
    return game_duration, game_mode, participants_df

api_key = 'RGAPI-e028b775-f8aa-4754-a343-f798a9e72332'
summoner_id = 'YKfiig2deMN2x_h6ifhd6PhXDGCwhxlhxzqLv_I1Ca17pegJT3v39UIOF1lewJufghu5_pdRyH103g'


## Get the match ids

In [4]:
match_list = get_match_list(api_key, summoner_id, 0, 10)

for match in match_list:
    print(match)

BR1_2848114478
BR1_2848091450
BR1_2847587196
BR1_2847053435
BR1_2843172863
BR1_2842671445
BR1_2839902431
BR1_2839881285
BR1_2781568879
BR1_2771129978


In [23]:
match_info = get_match_data(match_list[3], api_key)
match_info_df = pd.DataFrame(match_info)

game_dur, game_md, df_part = extract_match_info(match_info)

df_part

,championName,kills,death,assists,totalDamageToChampions,goldEarned,visioScore,role,teamId
0,Gnar,4,3,1,30214,19312,28,TOP,100
1,Lillia,1,7,6,25110,15328,37,JUNGLE,100
2,Yasuo,12,10,4,44392,20347,29,MIDDLE,100
3,Zeri,10,9,5,50128,21029,29,BOTTOM,100
4,Janna,0,10,14,8265,10773,72,UTILITY,100
5,Gangplank,5,6,6,17004,17193,20,TOP,200
6,Ivern,5,8,19,23585,16433,26,JUNGLE,200
7,Ryze,7,7,6,33867,20850,21,MIDDLE,200
8,Darius,19,3,13,44216,22313,15,BOTTOM,200
9,Lulu,3,3,27,14694,12214,87,UTILITY,200


In [5]:
match_data_tst = get_match_data(api_key='RGAPI-e028b775-f8aa-4754-a343-f798a9e72332',match_id='BR1_2839902431')

print(match_data_tst)

{'metadata': {'dataVersion': '2', 'matchId': 'BR1_2839902431', 'participants': ['5duzlBHx44I9KwnRBcDgPdhwRTDs1xHJmhEKeD7SR4fT3mF--DajtHiKNOEJdtMVaskmuE_GgKttZQ', 'DguENxg1C-3kx3NYlZ_BMeA3umDin-dzrCi3H9X-92n63-3HCv3PHyw8kE83TvR6-S5QNiweS8fpNg', '81iyOi0XNRAUAL058FxhePvq_9YYwZ1fmyvELYWviAyZHhl41ZjWn4vVzSp65XFfs5ijHGAjOQbz8g', '3SOIusGyrrkQD3s8QX82ztkd4pk1uhdGiHfgKdqPGWrnbM70pgegx2pIPzoaTVj_5k3WQWgZ0RlqxA', 'YKfiig2deMN2x_h6ifhd6PhXDGCwhxlhxzqLv_I1Ca17pegJT3v39UIOF1lewJufghu5_pdRyH103g', '0s_zgYlQTDuk4-hmaVbL85JHFcHTrV009TiNlBGIaDiS6QXq-t5cWIe10GyY_c0y9TCaofbWnE2l-g', 'E6G1LgP8kAGB9jsCY40UKLQCcAeCl6IYl_2vuvc_8K8Gv0VmgFLO5ofm9TKIt2wravUqe-Dmadc8_A', 'dCJVK7ijjEb9J7LvpHs3Awp2XsVPAhDHPW8R8k1ok6dj6vsA-Ry6zkswlzHkKUj2slvIQ3O0B5RsUg', 'i7hAwr-wlCI8FpcwKMTPcQTc2XeMhvvlTRWCuNx3zHvTWJt847GPkZB5hExGRZkMi83VqvSol39ZKw', '9-M4mtJM0ZCgqK7MOH6j_KC5kqoVq7E6H7DGXi8dxxUc3ppi4cXvaMPfWxsZ_2fQbiiPHL29bYABbw']}, 'info': {'gameCreation': 1700364212153, 'gameDuration': 2786, 'gameEndTimestamp': 1700367077260, '